In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
import torch.utils.data as data
import mlflow
import mlflow.pytorch
import mlflow.models.signature
import numpy as np
from mlflow.models.signature import infer_signature
import os


/home/mlops/fashion/venv/lib/python3.12/site-packages/mlflow/utils/requirements_utils.py:20: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources  # noqa: TID251


In [2]:
mlflow.set_tracking_uri("http://localhost:5000")  # Use this since file-based didn’t work
mlflow.set_experiment("FashionMNIST-MLP")


<Experiment: artifact_location='file:///home/mlops/mlruns/3', creation_time=1753495053836, experiment_id='3', last_update_time=1753495053836, lifecycle_stage='active', name='FashionMNIST-MLP', tags={}>

In [3]:
params = {
    "learning_rate": 0.001,
    "batch_size": 64,
    "epochs": 5,
    "hidden_size": 128
}


In [4]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

train_dataset = datasets.FashionMNIST(root="./data", train=True, download=True, transform=transform)
test_dataset = datasets.FashionMNIST(root="./data", train=False, download=True, transform=transform)

train_loader = data.DataLoader(train_dataset, batch_size=params["batch_size"], shuffle=True)
test_loader = data.DataLoader(test_dataset, batch_size=params["batch_size"], shuffle=False)


In [5]:
class MLP(nn.Module):
    def __init__(self, hidden_size):
        super(MLP, self).__init__()
        self.model = nn.Sequential(
            nn.Flatten(),
            nn.Linear(28*28, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, 10)
        )

    def forward(self, x):
        return self.model(x)


In [6]:
def train(model, epochs, optimizer, train_loader):
    criterion = nn.CrossEntropyLoss()
    model.train()
    train_losses = []

    for epoch in range(epochs):
        total_loss = 0
        correct = 0
        total = 0

        for x, y in train_loader:
            optimizer.zero_grad()
            output = model(x)
            loss = criterion(output, y)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            _, predicted = torch.max(output.data, 1)
            total += y.size(0)
            correct += (predicted == y).sum().item()

        avg_loss = total_loss / len(train_loader)
        accuracy = correct / total

        train_losses.append(avg_loss)

        mlflow.log_metric("train_loss", avg_loss, step=epoch)
        mlflow.log_metric("train_accuracy", accuracy, step=epoch)
        print(f"Epoch {epoch+1}: Loss={avg_loss:.4f}, Accuracy={accuracy:.4f}")

    return model, train_losses


In [7]:
def evaluate(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for x, y in test_loader:
            output = model(x)
            _, predicted = torch.max(output.data, 1)
            total += y.size(0)
            correct += (predicted == y).sum().item()

    return correct / total


In [8]:
with mlflow.start_run(run_name="MLP_Fashion"):
    for k, v in params.items():
        mlflow.log_param(k, v)

    model = MLP(params["hidden_size"])
    optimizer = optim.Adam(model.parameters(), lr=params["learning_rate"])

    model, train_losses = train(model, params["epochs"], optimizer, train_loader)

    test_accuracy = evaluate(model, test_loader)
    mlflow.log_metric("test_accuracy", test_accuracy)

    # Dummy input/output for signature
    dummy_input = torch.randn(1, 1, 28, 28)
    model.eval()
    with torch.no_grad():
        dummy_output = model(dummy_input)

    signature = infer_signature(dummy_input.cpu().numpy(), dummy_output.cpu().numpy())

    mlflow.pytorch.log_model(
        model, artifact_path="model", signature=signature
    )

    print(f"✅ Test Accuracy: {test_accuracy:.4f}")


Epoch 1: Loss=0.5006, Accuracy=0.8204
Epoch 2: Loss=0.3772, Accuracy=0.8632
Epoch 3: Loss=0.3420, Accuracy=0.8743
Epoch 4: Loss=0.3152, Accuracy=0.8840
Epoch 5: Loss=0.2984, Accuracy=0.8905


2025/07/26 10:01:58 WARNING mlflow.utils.requirements_utils: Found torch version (2.5.0+cpu) contains a local version label (+cpu). MLflow logged a pip requirement for this package as 'torch==2.5.0' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
2025/07/26 10:02:01 WARNING mlflow.utils.requirements_utils: Found torch version (2.5.0+cpu) contains a local version label (+cpu). MLflow logged a pip requirement for this package as 'torch==2.5.0' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.


✅ Test Accuracy: 0.8742
